<a href="https://colab.research.google.com/github/MarriRohan/HIGH-PERFORMANCE-COMPUTING/blob/main/hpcass3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time, cProfile, pstats, io, tracemalloc

def gen_grid(h, w):
    return [[(i * j) % 255 / 255.0 for j in range(w)] for i in range(h)]

def conv2d(grid, kernel):
    H, W = len(grid), len(grid[0])
    kh, kw = len(kernel), len(kernel[0])
    rh, rw = kh // 2, kw // 2

    out = [[0.0] * W for _ in range(H)]

    for i in range(rh, H - rh):
        for j in range(rw, W - rw):
            acc = 0.0
            for di in range(-rh, rh + 1):
                for dj in range(-rw, rw + 1):
                    acc += grid[i + di][j + dj] * kernel[di + rh][dj + rw]
            out[i][j] = acc

    return out

def make_uniform_kernel(size=5):
    val = 1.0 / (size * size)
    return [[val] * size for _ in range(size)]

def main():
    H, W = 256, 256
    grid = gen_grid(H, W)
    kernel = make_uniform_kernel(5)

    tracemalloc.start()
    pr = cProfile.Profile()
    pr.enable()

    t0 = time.perf_counter()
    out = conv2d(grid, kernel)
    t1 = time.perf_counter()

    pr.disable()

    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    print(f"Grid: {H}x{W} | Kernel: 5x5 | Time: {t1 - t0:.3f} s")
    print(f"Sample out[128][128] = {out[128][128]:.6f}")
    print(f"Current/Peak memory: {current/1e6:.2f} MB / {peak/1e6:.2f} MB")

    s_buf = io.StringIO()
    ps = pstats.Stats(pr, stream=s_buf).sort_stats('cumtime')
    ps.print_stats(10)

    print("\n--- cProfile (Top 10 by cumulative time) ---")
    print(s_buf.getvalue())

if __name__ == "__main__":
    main()

Grid: 256x256 | Kernel: 5x5 | Time: 6.622 s
Sample out[128][128] = 0.490980
Current/Peak memory: 2.07 MB / 2.07 MB

--- cProfile (Top 10 by cumulative time) ---
         22 function calls (21 primitive calls) in 6.623 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        6    6.040    1.007    6.040    1.007 {built-in method time.sleep}
      2/1    0.583    0.291    0.370    0.370 /tmp/ipython-input-3800845047.py:6(conv2d)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        7    0.000    0.000    0.000    0.000 {built-in method posix.getppid}
        2    0.000    0.000    0.000    0.000 {built-in method time.perf_counter}
        4    0.000    0.000    0.000    0.000 {built-in method builtins.len}



